# Continuous Integration and Continuous Delivery

## Lesson Overview

By the end of this lesson you should understand what CI/CD is and a simple example of how it can be applied to your own codebase. In this lesson you will learn about the following topics:

* What CI/CD does and how automation of testing assists with maintaining code quality across large projects.
* How GitHub actions can be used to implement CI/CD workflows
* How CI/CD tests can be run on multiple backends in parallel with matrix builds
* Examples of CI/CD in action with a small test project as well as production-grade CI/CD workflows.

## Related Training Video

Video link will be posted after the session.

## Why is CI/CD important?

Continuous Integration and Continous Delivery (CI/CD) refers to an automation strategy for testing code within a codebase and then integrating the tested code into a product ready for use in a production environment. The primary benefit of CI/CD is that it allows for quick evaluation, integration, and deployment of a collaborative code base.

* **Continuous Integration** - refers to the automated process of testing and integrating new code. Typically with a GitHub based flow, code is tested from branches or remote forks and then integrated into the main code branch when it passes tests and code review. Packaging of the code into an executable format can also be part of the integration process.  
* **Continuous Delivery** - tested and approved code changes are staged and then pushed to the production environment.   
* **Continuous Deployment** - refers to a process where production-ready code is automatically deployed after extensive automated testing. This is less common and mostly used for very mature codebases with high code coverage.

![ci-cd-pipeline](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/git-ci-cd.png?raw=1)

The Figure shows the typical workflow for a CI/CD pipeline. 

Read more in depth discussion about CI/CD via [GitHub's overview](https://resources.github.com/devops/ci-cd/).

## CI/CD Basics

This training focuses on the use of [GitHub Actions](https://docs.github.com/en/actions), but there are several other popular CI/CD frameworks that you might encounter for the automation of code testing and deployment.

### Other CI Frameworks

There are many different CI/CD frameworks that you could use. Other common frameworks for CI/CD that you might encounter include:

- [Jenkins](https://www.jenkins.io/) - self-contained, open source, and Java based; typically used for on-premise deployments.
- [CircleCI](https://circleci.com/).
- [Maven](https://maven.apache.org/what-is-maven.html) - open source, Apache licensed Java Project Object Model framework.
- [Azure Pipelines](https://learn.microsoft.com/en-us/azure/devops/pipelines/architectures/devops-pipelines-baseline-architecture?view=azure-devops) - Microsofts DevOps framework that ties in with other MS Azure infrastructure
- [Gitlab CI/CD](https://docs.gitlab.com/ee/ci/) - integrated with Gitlab cloud instances and self-hosted Gitlab installations.

## Common CI/CD formats

Many CI/CD frameworks use a common configuration format to specify the steps of a particular CI/CD workflow, typically YAML.   

* To learn more about how GitHub Actions uses YAML see this [blog post](https://docs.github.com/en/actions/using-workflows/workflow-syntax-for-github-actions).  
* For a quick reference on how to specify certain parameters in YAML, see this [Learn YAML in X Minutes page](https://learnxinyminutes.com/docs/yaml/).
  
A simple YAML example for a GitHub Action that triggers on a `push` operation to the main branch might look as follows:  

```yaml
on:
  push:
    branches:
      - main
```

It is also possible to invoke an Action on every push to any branch, or specify multiple triggers at once:
```yaml
on: [push, pull_request]
```

# Investigating GitHub Actions

Before we get started, we should define some common terminology for the [components of GitHub Actions](https://docs.github.com/en/actions/learn-github-actions/understanding-github-actions#the-components-of-github-actions).

## Terminology

- **GitHub Actions** - the CI/CD infrastructure that supports build, test, and deploymen automation
- **Action** - A piece of code that supports a common and repeatable task via the GH Actions framework. Typically Docker, Javascript, or composite and can be published as a GitHub App.
- **Workflow** - An automated process that contains 1 or more jobs, usually written in a YAML file.
- **Runner** - A server that executes a single job from a workflow. GitHub hosted runners include Linux, Windows, and MacOS, or you can also set up [self-hosted runners](https://docs.github.com/en/actions/hosting-your-own-runners/managing-self-hosted-runners/about-self-hosted-runners).
- **Job** - A job is a set of steps that either runs a shell script or executes an action. Job steps are executed sequentially but separate jobs can be run in parallel using separate runners.

## GitHub Workflows

Workflows are typically specified in the top-level directory `.github/workflows`. We will incrementally improve a `test` YAML file from our [sample repo](https://github.com/gt-ospo/testing_ci_cd/) to investigate the pieces of a typical workflow.

---
**NOTE**

To follow along with this training, please fork the [testing-with-ci-cd repo](https://github.com/gt-ospo/testing-with-ci-cd) so you can modify the .yml files and check the outputs.

---

## A Simple Testing Workflow (V1)

The first version of our workflow, [test-v1.yml](https://github.com/gt-ospo/testing-with-ci-cd/blob/main/.github/workflows/test-v1.yml) shows a very basic Python-oriented workflow. Note that the job is named `test-v1`, it uses a GitHub supported runner based on an Ubuntu VM, and it executes several actions and steps to install a specific version of Python.

```yaml
name: Grade Project Test - V1
on:
  push:
    branches:
      - 'main'
jobs:
  test-v1: # <-- this job name is totally up to you
    runs-on: ubuntu-latest # <-- usually ubuntu-latest, windows-latest, or macos-latest
    steps:
      - uses: actions/checkout@v4 # <-- every job runs on a clean runner, so you almost always start a job by checking out the code
      - name: Set up Python
        uses: actions/setup-python@v5 # <-- this is a GitHub Marketplace action: https://github.com/actions/setup-python
        with:
          python-version: '3.10'
      - name: Display Python version
        run: python -c "import sys; print(sys.version)"

      #If you look at the top level of the repo, you will notice that it includes a requirements.txt to add the correct Python packages to your runner
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
```

Let's say we tweak the Python version to use Python 3.12 instead of 3.10 and we commit to the main branch. Looking at the Actions tab, we see that this test will then run because we pushed to the main branch.

![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-test-v1-simple-output.jpg?raw=1)

Looking more in-depth at the action, we can see the individual steps of the job within the workflow as well as if it completed correctly.

![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-test-v1-output.jpg?raw=1)

![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-test-v1-simple-detailed-output.jpg?raw=1)

### Hands-on Exercise 1

> [!NOTE]  
>  Update the Python version to 3.12 with `test-v1.yml`. Rerun the action and check the output.

## Adding Tests and Tools to a Workflow (V2)

## Running tests

Building on our tests that we developed using PyTest in the last lesson, we can now automate the running of these tests using our CI/CD workflows.

## Adding code linting

We can further improve our testing automation by adding a code linter job as part of another workflow file, [lint.yml](https://github.com/jyoung3131/testing-with-ci-cd/blob/main/.github/workflows/lint.yml).

This YML file is very similar to the testing job, but it uses [ruff](https://astral.sh/ruff) to lint the test code.

```yaml
name: Grade Project Lint
on:
  push:
jobs:
  lint:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.12'
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
      - name: Lint with Ruff
        run: |
          pip install ruff
          ruff --output-format=github .
        continue-on-error: true
```

Looking at the output from the action, we notice that the job completes successfully but the linter reports several warnings of unused imports and parameters in the test code that could be addressed to improve the code quality.
  
![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-lint-output.jpg?raw=1)

## Using Matrix Builds (V3)

[Matrix Builds](https://docs.github.com/en/actions/using-jobs/using-a-matrix-for-your-jobs) allow you to build on the modular nature of jobs where each job can be launched **in parallel** using multiple backends (Linux, MacOS, Windows) or versions of software (Python 3.X).   

Version 3 of our test YAML file includes the same testing as v2 of the workflow, but it also creates a matrix strategy to test multiple versions of Python with our codebase. Note that the runner, `ubuntu-latest` could also be updated with a matrix strategy to run on multiple different VM targets.

```yaml
name: Grade Project Test - V3
on:
  push:
    branches:
      - 'main'
jobs:
  test-v3:
    runs-on: ubuntu-latest # <-- usually ubuntu-latest, windows-latest, or macos-latest
    strategy:
      matrix: # <-- indicates we want to run this test job in a parametrized fashion, in this case specifying multiple Python versions
        python-version: ["3.9", "3.10", "3.11", "3.12"]
    steps:
      - uses: actions/checkout@v4

      - name: Set up Python ${{ matrix.python-version }}
        uses: actions/setup-python@v5
        with:
          python-version: ${{ matrix.python-version }}
      - name: Display Python version
        run: python -c "import sys; print(sys.version)"
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
      - name: Test with pytest
        run: |
          coverage run -m pytest
          coverage xml -o coverage-${{ matrix.python-version }}.xml
      - name: Upload pytest coverage results
        uses: actions/upload-artifact@v4
        with:
          name: pytest-results-${{ matrix.python-version }}
          path: coverage-${{ matrix.python-version }}.xml
        if: ${{ always() }}
```

The output from the matrix build   

![image](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-test-v3-output.jpg?raw=1)

### Hands-on Exercise 2

> [!NOTE]  
> Use the matrix strategy to add a run using either the macos-latest or windows-latest backend

# Useful GitHub Actions Features

#### Activating and Deactivating Workflows

As we add more workflows, we may want to [enable or disable a specific workflow](https://docs.github.com/en/enterprise-server@3.9/actions/using-workflows/disabling-and-enabling-a-workflow). We might do this to save time or resources or just because we need to debug one specific workflow within the project.  

As we see in the figure here, we can deactivate our older versions of the test workflow and just use the newest version, V3.  

![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-disable-workflow.jpg?raw=1)

![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-disable-workflow-2.jpg?raw=1)

# Deployment

Once you have implemented automated testing for your codebase, you can then move on to the "delivery" part of CI/CD, which involves deploying your code either as an executable, tarball, or package for download or as part of a larger production environment. For our codebase, we would like to deploy a new Python package to [PyPI](https://pypi.org/).

The [deploy.yml](https://github.com/jyoung3131/testing-with-ci-cd/blob/main/.github/workflows/deploy.yml) workflow file uses the `gh-action-pypi-publish` [action](https://github.com/pypa/gh-action-pypi-publish) to push our package to PyPI, but we need to examine two more advanced topics before looking at the deployment.

## Usage of Secrets

[GitHub Secrets](https://docs.github.com/en/actions/security-guides/using-secrets-in-github-actions) can be used to store sensitive data like keys or passwords that need to be used by a GitHub workflow. Secrets are specified at the repository level, and they can be shared with reviewers without revealing the sensitive data inside the secret.

![image.png](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-deployment-gh-secret.jpg?raw=1)

## Concurrency with GitHub Action Jobs

Jobs can typically run in parallel, but we might not want a deployment to run multiple, concurrent jobs especially if it is generating output artifacts. The `concurrency` keyword can be used with groups to limit the amount of parallelism for a particular job. In our deploy.yml, this looks as follows:  

```yaml
#Specifies that any job in this workflow can only run one at a time - cancel previously running jobs before starting a new one
concurrency:
  group: ${{ github.workflow }}
  cancel-in-progress: true
```

Read more about concurrency with GitHub Actions [here](https://docs.github.com/en/actions/using-jobs/using-concurrency). 

## Doing the Deployment  

Now that we understand GitHub secret variables and concurrency, we can look at the final [deploy.yml](https://github.com/jyoung3131/testing-with-ci-cd/blob/main/.github/workflows/deploy.yml) workflow file.

```yaml
name: Grade Package Upload

on:
  # lets make this one manual invocation
  # normally you would see a "release" trigger here or a "create: tags: ..." trigger because releases are usually tagged
  workflow_dispatch:

concurrency:
  group: ${{ github.workflow }}
  cancel-in-progress: true

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.x'
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install build
      - name: Build package
        run: python -m build
      - name: Publish package
        # ends up here: https://pypi.org/project/CI-CD-Demo-Grade-Project/
        uses: pypa/gh-action-pypi-publish@release/v1
        with:
          password: ${{ secrets.PYPI_API_TOKEN }}
```

This workflow is only triggered manually. You can see the output of this action by testing it out manually and checking out the uploaded package at [PyPI](https://pypi.org/project/CI-CD-Demo-Grade-Project/0.1/).

### Example Workflow: Open Telemetry

Some of our summer projects have more complicated workflows that we can look at like [OpenTelemetry](
https://github.com/open-telemetry/opentelemetry-python/tree/main/.github/workflows).

Looking at the [test_0.yml file](https://github.com/open-telemetry/opentelemetry-python/blob/main/.github/workflows/test_0.yml), we can notice several features that we have explored earlier:

1. The usage of actions to checkout code and install Python
2. The installation of tox for testing automation
3. The usage of matrix strategies to test different packages, Python, and OS configurations

----

#### Learn More
Other example GitHub Action pipelines:
- [Open Horizon Pipelines](https://github.com/open-horizon/anax/tree/master/.github/workflows)
- [Open Horizon Web Pipeline](https://github.com/open-horizon/open-horizon.github.io/tree/master/.github/workflows)

# Self-hosted Runners    
A [self-hosted runner](https://docs.github.com/en/actions/hosting-your-own-runners/managing-self-hosted-runners/about-self-hosted-runners) is a GitHub runner that connects to a local service or daemon on a local server or VM. A self-hosted runner is useful if you need to test some target HW/SW that is not easily available via GitHub, you need to test on your local production platform, or if you don't want to use the cloud to run with your code or data.
  
The Spatter benchmark suite uses a self-hosted runner to test the CUDA backend with a recent NVIDIA GPU, and the relevant portion of its build.yml file contains the following:  

```yaml
  build-cuda:
    runs-on: self-hosted
    steps:
    - uses: actions/checkout@v4
    - name: Run batch file
      run: cd tests/misc && chmod +x run-crnch-cuda.sh && sbatch run-crnch-cuda.sh
```  
This runner is set up under the [Actions->runner tab for the project](https://github.com/hpcgarage/spatter/actions/runners), and when a push to the main branch is triggered it connects to the self-hosted service and runs the GitHub job on a node with an NVIDIA GPU.

![image](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-self-hosted-runner-cuda.jpg?raw=1)  

![image](https://github.com/gt-ospo/oss-training/blob/main/img/lesson-04/ci-cd-self-hosted-runner-cuda-output.jpg?raw=1)

### (Optional) Hands On Practice

  * Test adding a self-hosted runner for a GPU or Arm architecture using documentation for the [CRNCH Rogues Gallery](https://gt-crnch-rg.readthedocs.io/en/main/general/ci-runners.html)!
  * This is a challenging exercise because it requires you to be comfortable with setting up GitHub CI/CD pipelines, creating a self-hosted runner, and testing it with a remote cluster.

### Hands On Practice 2

Fork the codebase and do the following:  

1. Update the Python version to 3.12 with `test-v1.yml`. Run the action and check the output.
2. Update the matrix for `test-v3.yml` to build on window-latest and macos-latest.
3. Invoke `lint.yml` from `test.yml`.
4. Investigate and learn more about using self-hosted runners.

## Quiz

1. Which of the following stages are not included in the continuous integration stage?
   1. Run tests
   2. Build code
   4. Stage release
   5. Create artifact

In [ ]:
# Input the answer below as a string, such as 'A', 'B', 'C', ...
a1 = None
### BEGIN SOLUTION
a1_sol = 'C'
### END SOLUTION

2. Continuous Deployment is a necessary stage in any software project's production release pipeline. 

In [ ]:
# Input the answer below as a Python boolean.
a2 = None
### BEGIN SOLUTION
a2_sol = False
### END SOLUTION

3. Which of the following is not a CI/CD framework?
   1. Maven
   2. Jenkins
   3. Selenium
   4. Azure Pipelines

In [1]:
# Input the answer below as a string, such as 'A', 'B', 'C', ...
a3 = None
### BEGIN SOLUTION
a3_sol = 'C'
### END SOLUTION

4. CI/CD pipelines are normally initiated by the programmer manually running a pipeline script within the project repository.

In [ ]:
# Input the answer below as a Python boolean.
a4 = None
### BEGIN SOLUTION
a4_sol = False
### END SOLUTION

5. What is a workflow in the context of an open-source project's CI/CD pipeline?
   1. The infrastructure needed to run a CI/CD pipeline.
   2. An automated process that runs 1 or more jobs.
   3. Any file that is written in YAML.
   4. A program that connects to a local service or daemon on a local server or VM to run jobs.

In [ ]:
# Input the answer below as a string, such as 'A', 'B', 'C', ...
a5 = None
### BEGIN SOLUTION
a5_sol = 'B'
### END SOLUTION

6. Which of the following is a YAML file that runs a GitHub Action job on every code push to the main branch to print the message `Hello World`?
   1. ```yaml
        job: Hello world
        on:
            push:
                branches:
                - "main"
        actions:
            deploy:
                runs-on: ubuntu
                steps:
                    - uses: actions/checkout@v4
                    - name: Say hi!
                      run: echo "Hello World"
       ```
   2. ```yaml
       name: Hello world
       on:
           push:
       jobs:
           deploy:
               runs-on: ubuntu
               steps:
                  - uses: actions/checkout@v4
                  - name: Say hi!
                    run: |
                      echo "Hello World"
       ```
   3. ```yaml
       name: Hello world
       on:
           push:
               branches:
                - "main"
       actions:
           deploy:
               runs-on: ubuntu
               steps:
                  - uses: actions/checkout@v4
                  - name: Say hi!
                    run: |
                      echo "Hello World"
       ```
   4. ```yaml
       name: Hello world
       on:
           push:
               branches:
                - "main"
       jobs:
           deploy:
               runs-on: ubuntu
               steps:
                  - uses: actions/checkout@v4
                  - name: Say hi!
                    run: |
                      echo "Hello World"
       ```

In [ ]:
# Input the answer below as a string, such as 'A', 'B', 'C', ...
a6 = None
### BEGIN SOLUTION
a6_sol = 'D'
### END SOLUTION

7. Which of the following is a benefit specific to matrix builds?
   1. Matrix builds allow the automated running of multiple concurrent jobs.
   2. Matrix builds allow a way to easily create an automated job for linting code.
   3. Matrix builds allow a way to launch jobs in parallel using multiple backends.
   4. Matrix builds allow the programmer to limit the amount of parallelism for a particular job.  

In [ ]:
# Input the answer below as a string, such as 'A', 'B', 'C', ...
a7 = None
### BEGIN SOLUTION
a7_sol = 'C'
### END SOLUTION

8. Check out the Open-Telemetry-Python repo's workflow file [prepare-release-branch.yml](https://github.com/open-telemetry/opentelemetry-python/blob/main/.github/workflows/prepare-release-branch.yml). What are the three main stages of this workflow?
   1. Install and verify prerequisites, create pull request against release branch, create pull request against main branch
   2. Checkout commit, create pull request against release branch, create pull request against main branch
   3. Install and verify prerequisites, checkout commit, create pull request against main branch
   4. Create new branch, checkout commit, create pull request against release branch

In [ ]:
# Input the answer below as a string, such as 'A', 'B', 'C', ...
a8 = None
### BEGIN SOLUTION
a8_sol = 'A'
### END SOLUTION

9. An action can be invoked for multiple triggers using YAML's array syntax.

In [ ]:
# Input the answer below as a Python boolean.
a9 = None
### BEGIN SOLUTION
a9_sol = True
### END SOLUTION

10. Read through the following blog post ["Debug Github Action"](https://danielabaron.me/blog/debug-github-action/). Which of the ways below can you use to debug your GitHub action workflow YAML file?
    1. On GitHub, go to the Actions tab of your repo to see which stages have successfully completed.
    2. Make sure that the YAML file is formatted correctly using a linting tool.
    3. Look at the log files which are also shown in the Actions tab.
    4. Use some Action debugger tool such as the Tmate debugger which allows you to ssh into the runner machine.
    5. A, B, C
    6. A, C, D
    7. All of the above

In [ ]:
# Input the answer below as a Python boolean.
a10 = None
### BEGIN SOLUTION
a10_sol = 'F'
### END SOLUTION

In [ ]:
### BEGIN HASHED AUTOTEST
assert a1 == a1_sol
assert a2 == a2_sol
assert a3 == a3_sol
assert a4 == a4_sol
assert a5 == a5_sol
assert a6 == a6_sol
assert a7 == a7_sol
assert a8 == a8_sol
assert a9 == a9_sol
assert a10 == a10_sol
### END HASHED AUTOTEST